In [1]:
import os
os.chdir("..")
print(os.getcwd())

/Users/andreaspletschko/Uni/5. Semester (HKUST)/Machine_Learning/COMP5212


In [2]:
MODEL_PATH = "checkpoints/model_ncg02.hpc.itc.rwth-aachen.de_20231122-073926_20_epochs_.pth"
TSV_PATH = "data/kaggle_dataset/train.tsv"
TEST_IMG2 = "data/kaggle_dataset/Turkey/canvas_1630047432.jpg"
TEST_IMG = "data/kaggle_dataset/Australia/canvas_1629272439.jpg"

In [3]:
from PIL import Image
import torch
from torchvision import transforms
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget, ClassifierOutputSoftmaxTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
import cv2

from src.models.model import get_model

/Users/andreaspletschko/.local/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
def load_model(model_path):
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model = get_model(model_name="mobilenet_v3_small", num_classes=56)
    model.load_state_dict(state_dict)
    model.eval()
    return model

def get_idx_to_label(tsv_path):
    labels = []
    tsv_file = open(tsv_path, "r")

    for line in tsv_file:
        if line in ['\n', '\r\n']:
            continue
        label, _ = os.path.split(line.strip())
        labels.append(label)

    idx_to_label = {idx: label for idx, label in enumerate(set(labels))}
    return idx_to_label

def load_img(image_path):
    transform = transforms.Compose([
        transforms.Resize((1536, 662)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    image = Image.open(image_path).convert("RGB")
    image_transformed = transform(image)
    return image_transformed

In [12]:
idx_to_label = get_idx_to_label(TSV_PATH)
label_to_idx = {label: idx for idx, label in idx_to_label.items()}

model = load_model(MODEL_PATH)
#print(model)
image = load_img(TEST_IMG)

target_layers = model.features[-1]
target_layers = model.classifier
for param in target_layers.parameters():
            param.requires_grad = True
input_tensor = image.unsqueeze(0)


# Construct the CAM object once, and then re-use it on many images:
cam = AblationCAM(model=model, target_layers=target_layers, use_cuda=False)
target_category = label_to_idx["data/kaggle_dataset/Turkey"]
targets = [ClassifierOutputSoftmaxTarget(2)]

grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
print(grayscale_cam.shape)
grayscale_cam = grayscale_cam[0, :]
image = image.permute(1, 2, 0).numpy()
visualization = show_cam_on_image(image, grayscale_cam, use_rgb=True)

/Users/andreaspletschko/opt/miniforge3/envs/ml_project/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/andreaspletschko/opt/miniforge3/envs/ml_project/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/18 [00:00<?, ?it/s]


IndexError: too many indices for tensor of dimension 2

In [11]:
print(f"visualization shape: {visualization.shape}")
cam_image = cv2.cvtColor(visualization, cv2.COLOR_RGB2BGR)
print(f"cam_image shape: {cam_image.shape}")
cv2.imwrite("blub.png", cam_image)

visualization shape: (1536, 662, 3)
cam_image shape: (1536, 662, 3)


True